# 01. extract image featrues

In [1]:
IMAGE_PATH = "../data/sample/image_path.txt"         # input
IMAGE_FEATURE = "../data/sample/image_features.pkl"  # output

FEATURE_EXTRACTOR_BATCH_SIZE = 8
DETECTRON_MODEL  = "../data/detection/detectron_model.pth"
DETECTRON_CONFIG = "../data/detection/detectron_config.yaml"

In [2]:
import sys
sys.path.append("../vilbert")
from extract_features_custom import FeatureExtractor

feature_extractor = FeatureExtractor(model_file    = DETECTRON_MODEL,
                                     config_file   = DETECTRON_CONFIG,
                                     batch_size    = FEATURE_EXTRACTOR_BATCH_SIZE,
)


====================== EVALUATION CONFIGURATION ======================
Summary
----------------------------------------------------------------------
 - model_file                         : ../data/detection/detectron_model.pth
 - config_file                        : ../data/detection/detectron_config.yaml
 - batch_size                         : 8
 - num_features                       : 100
 - output_folder                      : 
 - image_dir                          : 
 - feature_name                       : fc6
 - confidence_threshold               : 0
 - background                         : False
 - partition                          : 0


In [3]:
with open(IMAGE_PATH) as f:
    image_path = f.readlines()

image_path = [x.strip() for x in image_path]
img_features = feature_extractor.extract_features_direct(image_path)

assert len(image_path) == len(img_features)

1it [00:02,  2.08s/it]


In [4]:
import pickle
with open(IMAGE_FEATURE, "wb") as f:
    pickle.dump(img_features, f)

## 02. Compute VilbertScore

In [1]:
IMAGE_FEATURE = "../data/sample/image_features.pkl"  # output from step 01
GENERATED_CAPTION = "../data/sample/gen_captions.txt"
GT_CAPTION = "../data/sample/gt_captions.txt"

VS_BATCH_SIZE = 8

In [2]:
import pandas as pd
import sys
sys.path.append("../vilbert")
import pickle
from compute_vilbertscore_custom import VilbertScore
vs = VilbertScore(batch_size=VS_BATCH_SIZE)

In [3]:
with open(IMAGE_FEATURE, "rb") as f:
    imgs = pickle.load(f)
            
with open(GENERATED_CAPTION, "r") as f:
    cand_caps = f.readlines()
    cand_caps = [x.strip() for x in cand_caps]

with open(GT_CAPTION, "r") as f:
    gt_caps = f.readlines()
    gt_caps = [x.strip() for x in gt_caps]

In [4]:
# load dataset to compute
vs.loaddata(list_image_feature = imgs,
            list_gen_caption = cand_caps,
            list_gt_caption = gt_caps,
            max_len=50
           )

In [6]:
# compute the score
precision, recall, f1 = vs.compute()
df = pd.DataFrame(data=[precision, recall, f1]).T
df = df.rename(columns={0:"precision", 1:"recall", 2:"f1"})
df

100%|██████████| 1/1 [00:00<00:00, 14.55it/s]

target data
Generated Captions (5 samples):  ['A man is skating on a boat.', 'Two girls are standing on a store.']
Ground truth Captions (5 samples):  ['A man is skating on a rail.', 'Two men in ethnic dress standing in a barren landscape.']


,precision,recall,f1
0,0.915213,0.916878,0.916045
1,0.875579,0.842759,0.858855
